In [1]:
from functools import *
import random

In [2]:
# Se le pasa una lista que define la permutación
def pi_inverse(l):
    n = len(l)
    Bn = BraidGroup(n)
    p = PermutationGroupElement(l)
    ciclos = p.cycle_tuples()
    b = Bn([1,-1])
    
    
    # Descomponemos en transposiciones los ciclos
    desc_trans = []
    for i in range(len(ciclos)):
        for j in range(len(ciclos[i])-1,0, -1):
            desc_trans.append((ciclos[i][0], ciclos[i][j]))
    
    desc_trans_sim = []
    for i in range(len(desc_trans)):
        d = [(desc_trans[i][0],desc_trans[i][0]+1)]
        for j in range(desc_trans[i][0]+1,desc_trans[i][1]):
            d = [(j,j+1)] + d + [(j,j+1)]
        desc_trans_sim = desc_trans_sim + d
            
    for i in range(len(desc_trans_sim)-1,-1,-1):          
                b = b * Bn([desc_trans_sim[i][0]])

        
    return b
    
    

In [3]:
def order_cycles(l):
    mov = True

    while mov:
        mov = False
        for i in range(len(l)-1):
            if len(l[i]) < len(l[i+1]):
                aux = l[i]
                l[i] = l[i+1]
                l[i+1] = aux
                mov = True
    
    return l

In [4]:
def GuessPermutation(v,w,n):
    vp = PermutationGroupElement(v.permutation()).cycle_tuples(singletons=True)
    wp = PermutationGroupElement(w.permutation()).cycle_tuples(singletons=True)
    
    vp = order_cycles(vp)
    wp = order_cycles(wp)

    
    tau = [i for i in range(n)]
    
    for i in range(len(wp)):
        for j in range(len(wp[i])):
            tau[vp[i][j]-1] = wp[i][j]
    #print(tau)
    return pi_inverse(tau)
    

In [5]:
Bn = BraidGroup(5)
v = Bn([1])
x = Bn([3])
w = (x^-1)*v*x
GuessPermutation(v,w,5)


1

In [6]:
def inf(b_normal,n):
    Bn = BraidGroup(n)
    alpha = Bn([1,-1])
    delta = Bn.delta()
    neutro = Bn([1,-1])

    
    rb = 0
    
    delta_r = b_normal[0]
    #print(delta_r)
    while delta_r != neutro:
        delta_r = delta_r*delta
        rb = rb + 1
        #print("rb", rb)
    #print("SALE")
    
    return rb
    

In [7]:
def tauC(w,r,n):
    Bn = BraidGroup(n)
    delta = Bn.delta()
    
    return (delta^-r)*w*(delta^r)

In [8]:
def compact(normal_form, n):
    Bn = BraidGroup(n)
    c = Bn([1,-1])
    
    for i in range(len(normal_form)):
        c = c*normal_form[i]
    
    return c
    


In [9]:
def breakConjugate(v,w,n):
    Bn = BraidGroup(n)
    delta = Bn.delta()
    alpha = Bn([1,-1])
    v_normal = v.left_normal_form()
    w_normal = w.left_normal_form()
    iteraciones = 0
    max_iteraciones = 100


    
    rw = inf(w_normal,n)
    rv = inf(v_normal,n)
    
    
    while rw < rv and iteraciones < max_iteraciones:
        gamma = delta*tauC(w_normal[1],rw,n)
        alpha = gamma*alpha
        w = gamma*compact(w_normal,n)*(gamma^-1)
        w_normal = w.left_normal_form()
        rw = inf(w_normal,n)
        iteraciones += 1



    sw = len(w_normal)-1 + inf(w_normal,n)
    sv = len(v_normal)-1 + inf(v_normal,n)
    while sw > sv and iteraciones < max_iteraciones:
        gamma = w_normal[len(w_normal)-1]
        alpha = gamma*alpha
        w = gamma*compact(w_normal,n)*(gamma^-1)
        w_normal = w.left_normal_form()
        sw = len(w_normal)-1 + inf(w_normal,n)
        iteraciones += 1
    
    v = compact(v_normal,n)
    w = compact(w_normal,n)
    mu = GuessPermutation(v,w,n)
    alpha = mu*alpha
    w = mu*w*(mu^-1)
    
    if v == w and iteraciones < max_iteraciones:
        return alpha
    else:
        return "failed"
        
        
        
    
    
    
    
    

In [10]:
Bn = BraidGroup(5)
v = Bn([-1,-3,2,4,-1,-3])
x = Bn([2,4,3,1,3])
w = (x^-1)*v*x
conjugator = breakConjugate(v,w,5)
#print(w == conjugator^-1*v*conjugator)
print("w: ", w.left_normal_form())
print("v: ", v.left_normal_form())
print("x: ", x)
print("conjugador: ", conjugator)

print(GuessPermutation(v,w,5))
print(w == conjugator^-1*v*conjugator)
#print(w.left_normal_form())
#print((conjugator^-1*v*conjugator).left_normal_form())

w:  ((s0^-1*s1^-1*s2^-1*s3^-1*s0^-1*s1^-1*s2^-1*s0^-1*s1^-1*s0^-1)^3, s0*s1*s2*s1*s0*s3*s2*s1*s0, s1*s0*s3*s2, s0*s2, s0*s1*s2*s1*s0*s3*s2*s1, s1*s0*s3*s2, s2)
v:  ((s0^-1*s1^-1*s2^-1*s3^-1*s0^-1*s1^-1*s2^-1*s0^-1*s1^-1*s0^-1)^2, s1*s0*s2*s1*s3*s2*s1*s0, s0*s2, s0*s1*s2*s1*s0*s3*s2*s1)
x:  s1*s3*s2*s0*s2
conjugador:  s1*s0*s3*s2^2
s1
True


In [21]:
def test(lv, lx, n, repeticiones):
    Bn = BraidGroup(n)
    index_generators1 = list(range(-n+1,0)) + list(range(1,n))
    index_generators2 = list(range(1,n))
    
    aciertos = 0
    
    for i in range(repeticiones):
        print("Progreso:", i*100/repeticiones)
        v_list = random.choices(index_generators1, k = lv)
        x_list = random.choices(index_generators2, k = lx)
        #print("v_list: ", v_list)
        #print("x_list: ", x_list)
        v = Bn(v_list)
        x = Bn(x_list)
       
        w = (x^-1)*v*x
        
        a = breakConjugate(v,w,n)
        
        if a != "failed":
            if (a^-1)*v*a != w:
                print("ERROR")
        
        if breakConjugate(v,w,n) != "failed":
            aciertos = aciertos + 1
    
    print("Aciertos: ", aciertos)
    print("Fallos: ", repeticiones-aciertos)
    

In [22]:
test(5,10,5,100)

Progreso: 0
Progreso: 1
Progreso: 2
Progreso: 3
Progreso: 4
Progreso: 5
Progreso: 6
Progreso: 7
Progreso: 8
Progreso: 9
Progreso: 10
Progreso: 11
Progreso: 12
Progreso: 13
Progreso: 14
Progreso: 15
Progreso: 16
Progreso: 17
Progreso: 18
Progreso: 19
Progreso: 20
Progreso: 21
Progreso: 22
Progreso: 23
Progreso: 24
Progreso: 25
Progreso: 26
Progreso: 27
Progreso: 28
Progreso: 29
Progreso: 30
Progreso: 31
Progreso: 32
Progreso: 33
Progreso: 34
Progreso: 35
Progreso: 36
Progreso: 37
Progreso: 38
Progreso: 39
Progreso: 40
Progreso: 41
Progreso: 42
Progreso: 43
Progreso: 44
Progreso: 45
Progreso: 46
Progreso: 47
Progreso: 48
Progreso: 49
Progreso: 50
Progreso: 51
Progreso: 52
Progreso: 53
Progreso: 54
Progreso: 55
Progreso: 56
Progreso: 57
Progreso: 58
Progreso: 59
Progreso: 60
Progreso: 61
Progreso: 62
Progreso: 63
Progreso: 64
Progreso: 65
Progreso: 66
Progreso: 67
Progreso: 68
Progreso: 69
Progreso: 70
Progreso: 71
Progreso: 72
Progreso: 73
Progreso: 74
Progreso: 75
Progreso: 76
Progreso:

In [13]:
test(5,5,5,100)

Progreso: 0
Progreso: 1
Progreso: 2
Progreso: 3
Progreso: 4
Progreso: 5
Progreso: 6
Progreso: 7
Progreso: 8
Progreso: 9
Progreso: 10
Progreso: 11
Progreso: 12
Progreso: 13
Progreso: 14
Progreso: 15
Progreso: 16
Progreso: 17
Progreso: 18
Progreso: 19
Progreso: 20
Progreso: 21
Progreso: 22
Progreso: 23
Progreso: 24
Progreso: 25
Progreso: 26
Progreso: 27
Progreso: 28
Progreso: 29
Progreso: 30
Progreso: 31
Progreso: 32
Progreso: 33
Progreso: 34
Progreso: 35
Progreso: 36
Progreso: 37
Progreso: 38
Progreso: 39
Progreso: 40
Progreso: 41
Progreso: 42
Progreso: 43
Progreso: 44
Progreso: 45
Progreso: 46
Progreso: 47
Progreso: 48
Progreso: 49
Progreso: 50
Progreso: 51
Progreso: 52
Progreso: 53
Progreso: 54
Progreso: 55
Progreso: 56
Progreso: 57
Progreso: 58
Progreso: 59
Progreso: 60
Progreso: 61
Progreso: 62
Progreso: 63
Progreso: 64
Progreso: 65
Progreso: 66
Progreso: 67
Progreso: 68
Progreso: 69
Progreso: 70
Progreso: 71
Progreso: 72
Progreso: 73
Progreso: 74
Progreso: 75
Progreso: 76
Progreso:

In [14]:
Bn = BraidGroup(5)
v = Bn([4, 1, 1, 4, -4])
x = Bn([3, 3, 4, 2, 1])
w = (x^-1)*v*x

breakConjugate(v,w,5)


'failed'

In [15]:
inf(Bn([3, 3, 4, 2, 1]).left_normal_form(),5)

0